<a href="https://colab.research.google.com/github/ashish-augustine/ashish-augustine/blob/main/evaluating_hint_genration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

## Loading the model from the hub

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


max_seq_length = 2048
dtype = None  # Auto
load_in_4bit = True
if True:

    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "ashish-augustine123/llama_hint_generation", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction: REPLACE THIS WITH YOUR PROMPT/QUESTION TO TEST THE MODEL ------
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.12: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Unsloth 2024.12.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What is a famous tall tower in Paris?

### Input:


### Response:
['It is a famous landmark of the city.', 'It was built in 1889 to celebrate the centenary of the French Revolution.', 'It is 324 meters (1,063 ft) tall.', 'It is the tallest structure in Paris and the second tallest in France.', 'It has been the world's tallest man-made structure for 41 years.', 'It has 2,500 steps that lead to the top.', 'The view from the top is breathtaking.', 'It is open to the public and is a popular tourist attraction.']<|end_of_text|>


In [ ]:
# Download the dataset from the hub
from datasets import load_dataset
dataset = load_dataset("ashish-augustine123/llama_hint_genration_data")
print(dataset)
test_dataset = dataset['test']
print(test_dataset)

README.md:   0%|          | 0.00/568 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/31.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/2.19M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/2.14M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/43920 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Question', 'Hints', 'text'],
        num_rows: 43920
    })
    validation: Dataset({
        features: ['Question', 'Hints', 'text'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['Question', 'Hints', 'text'],
        num_rows: 3000
    })
})
Dataset({
    features: ['Question', 'Hints', 'text'],
    num_rows: 3000
})


In [ ]:
# test_dataset = test.map(formatting_prompts_func, batched=True, batch_size=1000)

def clean_output(response_list):
    """Cleans and combines list-style outputs into a cohesive text."""
    text = " ".join(set(response_list))  # Remove duplicates
    text = text.replace("[^1^]", "").strip()  # Remove references
    return text

def evaluate_model(model, tokenizer, dataset, max_samples=10):
    """Evaluate model and produce cleaned outputs."""
    outputs = []
    for i, example in enumerate(dataset.shuffle().select(range(max_samples))):
        question = example["Question"]
        inputs = tokenizer([alpaca_prompt.format(question, "", "")], return_tensors="pt").to("cuda")
        raw_outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
        raw_hint = tokenizer.decode(raw_outputs[0], skip_special_tokens=True)
        cleaned_hint = clean_output(raw_hint.split("."))
        outputs.append({"Question": question, "Hint": cleaned_hint})

    return outputs

evaluated_outputs = evaluate_model(model, tokenizer, test_dataset)
for output in evaluated_outputs:
    print(f"Question: {output['Question']}\nHint: {output['Hint']}\n")
